# wdists

In [1]:
from ipynb.fs.full.koselleck import *

[Koselleck] (11:36:46) Alles bereit 


In [2]:
def vecs2dist(dfvecs,only_valid_words=True,lim=None):
    if only_valid_words:
        dfvecs=dfvecs.loc[
            set(dfvecs.index)&set(get_valid_words())
        ]
    if lim: dfvecs=dfvecs[:lim]
    dfsim=pd.DataFrame(
        fastdist.cosine_pairwise_distance(
            dfvecs.values.astype(float),
            return_matrix=True
        ),
        index=dfvecs.index,
        columns=dfvecs.index
    )
    dfdist=1-dfsim
    dfdist = dfdist.sort_index(axis=0)
    dfdist = dfdist.sort_index(axis=1)
    return dfdist

In [3]:
# X=vecs('1720-1740')
# # X
# o=vecs2dist(X)
# o

In [7]:
def do_vdist(obj,**y):
    corpus,period,run = obj if type(obj) in {list,tuple} else obj.split('/')
    dfvecs=vecs(corpus=corpus, period=period, run=run)
    dfdist=vecs2dist(dfvecs,**y)
    return (obj,dfdist)

def vdist(periods,run=None,dfmodels=None,
          corpus=DEFAULT_CORPUS,max_runs=MAX_RUNS,
          only_valid_words=True,lim=10000,
          progress=True,num_proc=1,force=False):
    corpora=to_words(corpus)
    periods=to_words(periods)
    if run is None: runs=[i+1 for i in range(max_runs)]
    else: runs=[run] if type(run)==int else run
    objs=sorted([(corpus,period,run) for corpus in corpora for period in periods for run in runs])
    key=lltk.hashstr(f'{objs}/only_valid_words={only_valid_words}__lim={lim}')[:11]
    
    objs_done={}
    def to_qstr(c,p,r):
        return f'{c}/{p}/{r}/only_valid_words={only_valid_words}__lim={lim}'
    
    if not force:
        with get_db('vdist','r') as db:
            if key in db: return db[key]
            for c,p,r in objs:
                objkey=to_qstr(c,p,r)
                if objkey in db:
                    objs_done|={obj}
    objs_todo=[obj for obj in objs if obj not in objs_done]
    
    def iterr():
        # iter todo
        yield from pmap_iter(
            do_vdist, objs_todo,
            num_proc=num_proc, progress=False,
            kwargs=dict(only_valid_words=only_valid_words,lim=lim),
        )
        # iter done
        for obj in objs_done:
            yield (obj,db[to_qstr(*obj)])
        
    df=pd.DataFrame()
    

    pbar=tqdm(iterr(),disable=not progress,desc='Merging distance matrices...',total=len(objs))
    for obj,dfdist in pbar:
#             pbar.set_description('Saving to db')
#             db[to_qstr(*obj)]=dfdist
#             pbar.set_description('Merging')
        df3=pd.concat([df,dfdist])
        try:
            df=df3.groupby(df3.index).mean()
        except Exception as e:
            print('!!',e,key)
    print('Saving to db')
    print(type(key), len(key), type(df), df.shape)
    with get_db('vdist','w') as db: db[key]=df
    return df    

In [11]:
# vdist('1700-1770',max_runs=10)

In [17]:
# !ls -ltrh ../db/
# !rm ../db/db.kos2.vdist.sqlite

In [18]:
vdist('1720-1725',max_runs=10)

[Koselleck] (11:31:47) ['39bd5d5c97ca52031d33dd4c1a853c69288cba91db18c709177c563e'] (+23.4s)


In [6]:
def do_ingest_dists(obj):
    key,dfperiod,progress,max_runs=obj
    df=pd.DataFrame()
    dfperiod=dfperiod[dfperiod.run_int<=max_runs]
    iterr=zip(dfperiod.corpus, dfperiod.period, dfperiod.run_int)
    if progress:
        iterr=tqdm(iterr,total=len(dfperiod),position=0)
    for corpus,period,run in iterr:
        dfvecs=vecs(corpus=corpus, period=period, run=run)
        dfdist=vecs2dist(dfvecs)
        df3=pd.concat([df,dfdist])
        try:
            df=df3.groupby(df3.index).mean()
        except Exception as e:
            print('!!',e,key)
                
    return (key,df)

In [7]:
# for i,dfprd in dfmodels.groupby(['corpus','period']): pass
# dfprd

In [8]:

def ingest_dists(dfmodels,num_proc=1,force=False, progress_data=True, max_runs=10):
    objs = [
        (f'/dists/{corpus}/{period}', gdf, progress_data, max_runs)
        for (corpus,period),gdf in dfmodels.groupby(
            ['corpus','period']
        )
    ]
    iterr=pmap_iter(
        do_ingest_dists,
        objs,
        desc='Ingesting distances into db...',
        num_proc=num_proc
    )
    
    with get_db('wdists',mode='a') as db:
        for key,odf in iterr:
            prefix,key2 = db_split_key(key)
            for i,wordcol in enumerate(odf.columns):
                qstr=f'{wordcol}/{key2}'
                db[qstr]=odf[wordcol]
                if i and not i%100: db.commit()
            db.commit()

In [9]:
#ingest_dists(dfmodels.query('period_start>=1900 & period_len==5'),num_proc=2,progress_data=False)
dfmodels=pd.DataFrame(get_model_paths()).sort_values('period_start')
dfmodels['period']=[f'{x}-{y}' for x,y in zip(dfmodels.period_start,dfmodels.period_end)]
dfmodels['run_int']=dfmodels.run.apply(lambda x: int(x.split('_')[-1]))
dfmodels=dfmodels.query('(period_end - period_start) == 20')
dfmodels1=dfmodels.query('period_start<1700')
dfmodels2=dfmodels.query('period_start>=1900')
# display(dfmodels1)
# display(dfmodels2)

Scanning directory for models: 3946it [00:00, 50068.59it/s]


In [10]:
# ingest_dists(dfmodels1,num_proc=2,progress_data=False)
# ingest_dists(dfmodels2,num_proc=1,progress_data=True)

In [11]:
def wdist(word_or_words, period_or_periods=None, corpus_or_corpora=DEFAULT_CORPUS,
          ybin=YBIN_WDIST, ymin=YMIN_WDIST, ymax=YMAX_WDIST, **attrs):
    words,corpora,periods = get_words_corpora_periods(
        word_or_words,period_or_periods,corpus_or_corpora,
        ybin=ybin,ymin=ymin,ymax=ymax
    )
    queries = [f'{w}/{c}/{p}' for w in words for c in corpora for p in periods]
    with get_db('wdists',mode='r') as db:
        d=dict(
            (
                qstr.replace('/','_'),
                db[qstr]
            )
            for qstr in queries
            if qstr in db and len(db[qstr])
        )
        odf=pd.DataFrame(dict((k,v) for k,v in d.items() if len(v)))
        odf['_avg_']=odf.mean(axis=1)
        return odf.sort_values('_avg_').drop('_avg_',1)

In [12]:
wdist('culture',ymin=1680, ymax=1960, ybin=20)

,culture_bpo_1680-1700,culture_bpo_1720-1740,culture_bpo_1740-1760,culture_bpo_1760-1780,culture_bpo_1780-1800,culture_bpo_1800-1820,culture_bpo_1820-1840,culture_bpo_1840-1860,culture_bpo_1860-1880,culture_bpo_1880-1900,culture_bpo_1900-1920,culture_bpo_1920-1940,culture_bpo_1940-1960
culture,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000
cultivation,NaN,0.471128,0.283059,0.189279,0.113374,0.107502,0.083780,0.115605,0.241122,0.327860,0.326516,0.443610,0.525450
growth,0.441376,0.504667,0.312152,0.338933,0.289719,0.278823,0.260696,0.260826,0.321123,0.379191,0.411677,0.533881,0.717367
cultivating,0.464214,0.329926,0.306642,0.214996,0.269945,0.285167,0.270587,0.354790,0.395134,0.429514,0.463855,0.536596,0.831495
agriculture,0.475700,0.360903,0.303680,0.293694,0.237383,0.265399,0.344214,0.387082,0.458024,0.561446,0.576507,0.625759,0.502106
...,...,...,...,...,...,...,...,...,...,...,...,...,...
beheaded,NaN,1.395286,1.295329,1.267372,1.242965,1.301845,1.292656,1.346799,1.323472,1.330551,1.266923,1.158228,1.131560
muttered,NaN,NaN,1.053747,1.255858,1.416219,1.461184,1.376880,1.317359,1.290660,1.237000,1.181595,1.171032,1.317398
shouted,NaN,NaN,NaN,1.228594,1.363689,1.402909,1.331414,1.297394,1.264934,1.211570,1.191744,1.220768,1.364025
telegram,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.207877,1.310399,1.342682,1.310011,1.310377,1.270344
